In [14]:
import os

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

print(tf.version.VERSION)

2.10.0


In [15]:
dataset = pd.read_csv('datas/final_data.csv')
dataset = pd.get_dummies(dataset, columns=['label'])

In [16]:
train_dataset = dataset.sample(frac=0.8, random_state=8) #train_dataset = 80% of total dataset  
#random_state = any int value means every time when you run your program you will get the same output for train and test dataset, random_state is None by default which means every time when you run your program you will get different output because of splitting between train and test varies within 
test_dataset = dataset.drop(train_dataset.index) #remove train_dataset from dataframe to get test_dataset

In [17]:
train_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    train_labels[x] = train_dataset.pop(x)

In [18]:
test_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    test_labels[x] = test_dataset.pop(x)

In [19]:
train_dataset = np.array(train_dataset)
train_labels = np.array(train_labels)
train_labels = np.argmax(train_labels, axis=1)

test_dataset = np.array(test_dataset)
test_labels = np.array(test_labels)
test_labels = np.argmax(test_labels, axis=1)


In [20]:
def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(3, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=(3,)))
    for i in range(1,10):
        model.add(tf.keras.layers.Dense(128, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
    model.add(tf.keras.layers.Dense(11))

    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.build()

    return model

In [21]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 3)                 12        
                                                                 
 dense_12 (Dense)            (None, 128)               512       
                                                                 
 dense_13 (Dense)            (None, 128)               16512     
                                                                 
 dense_14 (Dense)            (None, 128)               16512     
                                                                 
 dense_15 (Dense)            (None, 128)               16512     
                                                                 
 dense_16 (Dense)            (None, 128)               16512     
                                                                 
 dense_17 (Dense)            (None, 128)              

In [22]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 verbose=1,
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 save_freq=10)

# Train the model with the new callback
model.fit(train_dataset, 
          train_labels,  
          epochs=900,
          batch_size = 1000,
          validation_data=(test_dataset, test_labels),
          callbacks=[cp_callback], # Pass callback to training
          shuffle=True)  

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/900
5/5 [==============================] - 2s 87ms/step - loss: 3.2259 - accuracy: 0.2927 - val_loss: 2.8336 - val_accuracy: 0.3446
Epoch 2/900
5/5 [==============================] - 0s 16ms/step - loss: 2.7688 - accuracy: 0.3486 - val_loss: 2.6052 - val_accuracy: 0.4109
Epoch 3/900
5/5 [==============================] - 0s 14ms/step - loss: 2.5854 - accuracy: 0.4245 - val_loss: 2.4786 - val_accuracy: 0.4525
Epoch 4/900
5/5 [==============================] - 0s 13ms/step - loss: 2.4665 - accuracy: 0.4297 - val_loss: 2.3413 - val_accuracy: 0.4515
Epoch 5/900
5/5 [==============================] - 0s 13ms/step - loss: 2.3458 - accuracy: 0.4456 - val_loss: 2.2489 - val_accuracy: 0.4545
Epoch 6/900
5/5 [==============================] - 0s 14ms/step - loss: 2.2653 - accuracy: 0.4562 - val_loss: 2.1888 - val_accuracy: 0.5168
Epoch 7/900
5/5 [==============================] - 0s 14ms/step - loss: 2.1821 - accuracy: 0.4728 - val_loss: 2.1164 - val_accuracy: 0.4723
Epoch 8/900
5/5 [===

In [23]:
model.evaluate(test_dataset, test_labels, verbose=2)

32/32 - 0s - loss: 0.4655 - accuracy: 0.8663 - 55ms/epoch - 2ms/step


[0.4654940068721771, 0.8663366436958313]

In [24]:
# this saves the whole model including the loss function used, everythin etc.
save_path = "saved_models"
model.save(f"{save_path}/model1")

INFO:tensorflow:Assets written to: saved_models/model1\assets


In [25]:
# this loads the whole model saved
# verbose just means what do you want to see in the outputs(what metrics do you want to see etc.)
new_model = tf.keras.models.load_model(f"{save_path}/model1")
new_model.evaluate(test_dataset, test_labels, verbose=2)

32/32 - 1s - loss: 0.4655 - accuracy: 0.8663 - 1s/epoch - 32ms/step


[0.4654940068721771, 0.8663366436958313]